In [1]:
from hello.hello3 import HelloApp, NotLoggedInError, BadError
import time, datetime
import requests

In [2]:
def wait1(s):
    dend = (datetime.datetime.now() + datetime.timedelta(seconds=s)).strftime("%m/%d/%Y %I:%M:%S %p")
    tprint("Sleeping %d seconds, ending at %s" %(s,dend))
    start = time.time()
    end = start + s
    try:
        while time.time() < end:
            time.sleep(5)
    except KeyboardInterrupt:
        pass

def wait2(s):
    pass
wait = wait1

hours = 3600
minutes = 60
seconds = 1

def now():
    return datetime.datetime.now().strftime("%H:%M:%S")
    
class H():
    def __init__(self, ip):
        self.ip = ip
        self.h = HelloApp(ip)
        
    def call(self, f, *args, _message=True, **kw):
        s = "(" + ", ".join(map(repr, args)) + ", ".join("%s=%r" %(k,v) for k,v in kw.items()) + ")"
        m = "%s %s: %s%s" % (self.ip, now(), f.__name__, s)
        if _message:
            tprint(m)
        while True:
            try:
                rv = f(*args, **kw)
            except NotLoggedInError:
                self.h.login()
            except requests.exceptions.ConnectTimeout as e:
                tprint(str(e))
                tprint("Make sure you're on the right network!")
            except (BadError, IOError) as e:
                tprint(str(e))
            except Exception as e: 
                tprint(str(e))
            else:
                return rv

    def startbatch(self, name):
        if self.call(self.h.batchrunning):
            self.call(self.h.endbatch)
        return self.call(self.h.startbatch, name)

    def setdo(self, mode, n2_or_sp, o2=None):
        self.call(self.h.setdo, mode, n2_or_sp,o2)

    def doman(self, n2, o2):
        self.setdo(1, n2, o2)

    def doauto(self, sp):
        self.setdo(0, sp)

    def dooff(self):
        self.setdo(2, 0, 0)

    def phoff(self):
        self.setph(2, 0, 0)

    def setph(self, mode, co2, base=None):
        self.call(self.h.setph, mode, co2, base)

    def phauto(self, sp):
        self.setph(0, sp)

    def endbatch(self):
        self.call(self.h.endbatch)

    def agauto(self, sp):
        self.call(self.h.setag, 0, sp)

    def setmg(self, sp):
        self.call(self.h.setmg, 1, sp)
        
    def getphpv(self):
        return self.call(self.h.gpmv, _message=False)['ph']['pv']


# CO2 Ramp Up/Down + Base

In [3]:
def test_co2_up(h):
    h.startbatch("ip80 2.2")
    h.phauto(7.5)
    
    # ramp up to at least 7.45
    while h.getphpv() < 7.45:
        wait(1*minutes)
    
    # the procedure calls to wait for 3 hours for stability "at set point"
    # before proceeding. In leu of more complex logic, just wait 6 hours instead
    wait(6* hours)
    
    h.endbatch()
    
def test_co2_down(h):
    h.startbatch("ip80 2.3")
    h.phauto(7.0)
    
    # ramp up to at least 7.45
    while h.getphpv() > 7.05:
        wait(1*minutes)
    
    # the procedure calls to wait for 3 hours for stability "at set point"
    # before proceeding. In leu of more complex logic, just wait 6 hours instead
    wait(6* hours)
    
    h.endbatch()
    
def test_base(h):
    h.startbatch("ip80 2.5")
    h.doman(65, 0)
    wait(5* hours)
    h.endbatch()

In [4]:
import threading
class HThread(threading.Thread):
    def __init__(self, ip):
        self.ip = ip
        super().__init__(daemon=True)
    def run(self):
        tprint("%s %s: Connecting"%(self.ip, now()))
        h = H(self.ip)
        #tprint("%s %s: Beginning CO2 Ramp Up"%(self.ip, now()))
        #test_co2_up(h)
        #tprint("%s %s: Beginning CO2 Ramp Down"%(self.ip, now()))
        #test_co2_down(h)
        tprint("%s %s: Beginning Base Test"%(self.ip, now()))
        test_base(h)
        tprint("%s %s: Testing Completed"%(self.ip, now()))

In [7]:
class TP():
    def __init__(self):
        self.threads = set()
        self._started = False
    def add(self, *args, **kw):
        t = HThread(*args, **kw)
        if self._started:
            t.start()
        self.threads.add(t)
    def start(self):
        for t in self.threads:
            t.start()
    def join(self):
        active = self.threads.copy()
        while True:
            threads = active.copy()
            for t in threads:
                if not t.is_alive():
                    active.remove(t)
            if not active:
                break
            time.sleep(5*minutes)

In [8]:
import queue
tq = queue.Queue()
tsentinel = object()
def tprint(*args, **kw):
    tq.put((args, kw))
    
def twork():
    while True:
        args, kw = tq.get()
        if args is tsentinel:
            break
        print(*args, **kw)

In [9]:
tp = TP()
for ip in ("192.168.1.6", "192.168.1.9", "192.168.1.11"):
    tp.add(ip)
tp.start()
printer = threading.Thread(target=twork, daemon=True)
printer.start()

192.168.1.11 17:06:11: Connecting
192.168.1.11 17:06:11: Beginning Base Test
192.168.1.11 17:06:11: batchrunning()
192.168.1.6 17:06:11: Connecting
192.168.1.6 17:06:11: Beginning Base Test
192.168.1.6 17:06:11: batchrunning()
192.168.1.9 17:06:11: Connecting
192.168.1.9 17:06:11: Beginning Base Test
192.168.1.9 17:06:11: batchrunning()
192.168.1.9 17:06:12: startbatch('ip80 2.4')
192.168.1.6 17:06:12: startbatch('ip80 2.4')
192.168.1.11 17:06:13: startbatch('ip80 2.4')
192.168.1.9 17:06:13: setdo(1, 65, 0)
192.168.1.6 17:06:13: setdo(1, 65, 0)
192.168.1.11 17:06:13: setdo(1, 65, 0)
Sleeping 18000 seconds, ending at 07/26/2018 10:06:17 PM
Sleeping 18000 seconds, ending at 07/26/2018 10:06:17 PM
Sleeping 18000 seconds, ending at 07/26/2018 10:06:17 PM


In [ ]:
tq.put((tsentinel, None))

### 3L Analysis

In [ ]:
h = HelloApp('192.168.1.14')
p=''

def bt():
    import datetime; return datetime.datetime.now().strftime("%y%m%d")

def dl(bn):
    if isinstance(bn, str):
        b = call(h.getdatareport_bybatchname, bn)
    elif isinstance(bn, tuple):
        name, d1, d2 = bn
        b = call(h.getreport_bydate, 'process_data', d1, d2)
        bn = name
    fn = p+bn+" " + bt() + ".csv"
    with open(fn, 'wb') as f:
        f.write(b)
    return fn

In [ ]:
for s in ("IE21 1.3", "IE21 1.4"):
    dl(s)

In [ ]:
from officelib.xllib import *
from officelib.const import xlconst as xlc


def open_file(file):
    global xl, wb, ws, cells, cr
    try:
        xl
    except NameError:
        xl = Excel()
    wb = xl.Workbooks.Open(file)
    ws = wb.Worksheets(1)
    cells = ws.Cells
    cr = cells.Range

def elform(c, ref):
    a1 = c.GetAddress(0,0)
    a2 = ref.GetAddress(1,1)
    a3 = ref.Offset(0,2).GetAddress(1,1)
    form = "=(%s-%s)*%s"%(a1, a2, a3)
    return form
    
def mk_elapsed(c, ref, ins=True):
    
    if ins:
        c.Offset(1,2).EntireColumn.Insert()
    top = c.Offset(1,2)
    bot = c.End(xlc.xlDown).Offset(1,2)
    top.Value = elform(c, ref)
    r = cr(top, bot)
    top.AutoFill(r)
    r.NumberFormat = "0.00"
    
def chart_rng(c):
    x1 = c.Offset(1,2)
    x2 = x1.End(xlc.xlDown)
    y1 = c.Offset(1,3)
    y2 = y1.End(xlc.xlDown)
    return cr(x1, x2), cr(y1, y2)

def mkch(c,v):
    chart = CreateChart(ws, xlc.xlXYScatterLinesNoMarkers)
    mkser(chart, c,v)
    
def mkser(ch, c,v):
    x, y = chart_rng(c)
    CreateDataSeries(ch, x, y, v)
    
def fixaxes():
    for co in ws.ChartObjects():
        ax = co.Chart.Axes(xlCategory, xlPrimary)
        ax.MinimumScale = 0
        
def analf(f, save=True):
    open_file(f)
    return analyze(f, save)

In [ ]:
def analyze(fn, save=True):
    c1 = cells.Find("pHPV")
    ref = c1 = c1.Offset(2,1)
    mk_elapsed(c1, ref)
    ref.Offset(0, 2).Value = "24"
    mkch(c1,"pHPV")
    ch2 = CreateChart(ws, xlc.xlXYScatterLinesNoMarkers)
    mkser(ch2, c1, "pHPV")
    
    for v in "pHCO2ActualRequest(%)", "pHBaseDutyActual(%)":
        c = cells.Find(v).Offset(2,1)
        mk_elapsed(c, ref)
        mkch(c,v)
        mkser(ch2, c, v)
    fixaxes()
    if save:
        wb.SaveAs(fn.replace('.csv','.xlsx'), FileFormat=xlc.xlOpenXMLWorkbook)
    return wb

In [ ]:
def tofilename(bn):
    return p + bn + " " + bt() + ".csv"

In [ ]:
import os
files = [p+bn+" " + bt() + ".csv" for bn in ("IE21 1.3", "IE21 1.4")]
xl = Excel()
with screen_lock(xl):
    xl.DisplayAlerts = False
    for file in files: 
        analf(os.path.abspath(file))

    # these two are reference files generated by Erica running through the draft OQ
    p2 = "C:\\PBSCloudStation\\(2) R&D-Product Engineering\\IQ OQ\\"
    fns = "2018050114414412.csv", "2018050308540168.csv", 
    fns = [
        (fns[0], "7.2 to 7.5"),
        (fns[1], "7.5 to 7.0")
    ]

    for fn, s in fns:
        fp = p2 + fn
        wb = analf(fp, False)
        wb.SaveAs(p + ("OQ CO2 %s "%s)+ bt()+".xlsx", FileFormat = xlc.xlOpenXMLWorkbook)
    xl.DisplayAlerts = True
wbs = list(xl.Workbooks)

In [ ]:
for wb in wbs:
    wb.Worksheets(1).ChartObjects(2).Chart.SeriesCollection(2).AxisGroup = xlc.xlSecondary

In [ ]:
def phxtime(idx, a, b):
    wb = wbs[idx]
    for co in wb.Worksheets(1).ChartObjects():
        ch = co.Chart
        FormatAxesScale(ch, a, b)
        

        
def phctype(idx, typ):
    wb = wbs[idx]
    wb.Worksheets(1).ChartObjects(1).Chart.ChartType = typ

In [ ]:
phxtime(0, 2, 6)
phxtime(2, 2, 6)

In [ ]:
phxtime(2, 10, 14)

In [ ]:
phctype(0, xlc.xlXYScatterLines)
phctype(2, xlc.xlXYScatterLines)

In [ ]:
phxtime(1, 2,6)
phxtime(3, 10, 14)

In [ ]:
def bgetc(idx):
    return xl.Workbooks("IE21 CO2 Test 180607.xlsx").Worksheets(1).ChartObjects(idx).Chart

def bxfix(idx, a, b):
    ch = bgetc(idx)
    FormatAxesScale(ch, a, b)
    
def byfix(idx, a, b):
    ch = bgetc(idx)
    FormatAxesScale(ch, None, None, a, b)

In [ ]:
for co in xl.Workbooks(5).Worksheets(1).ChartObjects():
    ch = co.Chart
    FormatAxesScale(ch, 0, 6)

In [ ]:
byfix(1, 7.2, 7.6)
byfix(2, 7.2, 7.6)

In [ ]:
byfix(3, 6.9, 7.6)
byfix(4, 6.9, 7.6)

In [ ]:
dl("IE21 2.3")
wb = xl.Workbooks.Open(tofilename("IE21 2.3"))

In [ ]:
file = wb.Path + "\\" + wb.Name

In [ ]:
wb.Close()

In [ ]:
analf(file, True)

In [ ]:
xl.Visible=True

In [ ]:
file="C:\\Users\\natha\\Downloads\\fullbasedata.csv"
analf(file, True)

In [ ]:
import clipboard
clipboard.paste()

In [ ]:
file = os.path.join(xl.ActiveWorkbook.Path, xl.ActiveWorkbook.Name)

In [ ]:
xl.ActiveWorkbook.Close(False)
analf(file)

In [ ]:
file = "C:\\PBSCloudStation\\(2) R&D-Product Engineering\\IQ OQ\\2018050714325226.csv"
analf(file)

In [ ]:
xl.Visible=True